In [1]:
import sys
import os
import numpy as np
import random
import json
import string
import itertools
import math
from Crypto.Util.number import long_to_bytes as ltb, bytes_to_long as btl
from pwn import remote, process, log, args, debug, info, log as pwnLog
from tqdm import tqdm, trange
from matplotlib import pyplot as plt
from functools import reduce, cache
from base64 import b64encode, b64decode
from collections import Counter

In [2]:
# Uncomment below line to use sagemath module
from sage.all_cmdline import *


In [3]:
hint = [[8833677163, 7159466859734884050485160017085648949938620549936739498951806707835448713685207536552299918328868591349533273061478374089984223260577742322460362334647], [1762352339, 226021067407224282748442153993506422184559341973942542463611713009302649608941949660293486972516731321467369225717344439888178648461773300463], [6814325828, 145915445591160853098610646953738314537732696913127480076359637783667652244881400087606152610739138506056218199806589240306741950875956525839170443027], [7865890147, 1255960511416167089973436987379886082394930531153251392262351559661203914293720867397614316726175343133363293139291718249474745356688772183204229822751], [3446680058, 5293859406843167459297872689128502546567761548640003856519557803475599388573073027426285178678302790672452768542207529392596772806973985884693237], [5877771652, 15883583178415793156782570756223737797760371065858523945056072346852806064052610100332389954372845836435762293469821829936427366159434784004504398291], [5589586633, 7472281200056449019563455444999813482028446397663996508394508567670602924631065370355170602075256758870709465268255309886778027432655593535614166637], [1175276268, 518629639886914674796931012497083502361229856009622285824810204881645367508380387007577326543311405957619591605841895258801496781885398507], [3312651249, 2920072124198357353277671402963439479294095254775553378538026906919501392975483266953780010186413153114694525677661955925502702904273824951901573], [1690420045, 120969905638890571692249167310237577968012605711450331530578304692989016303379573026678222839813088165787719888874515256743894818676147474521], [8298141391, 2802013920829536770649820952830225273137583982204944734413323800249577243089166668778583649665043009034143120874987986020037964205143133245123290632883], [733386150, 439287044309927586596972381366960178061704411347096135895831191742005839221734048948610767236121358802659929070752762370822244956535801], [7897145685, 1332938401210287323326359805632057169759318295533885927320250339098837407040892547133970478663396358868892779722453565866390506758764909670000617998161], [9797888335, 33864534898740204255025855638155912349784294672865719351405048784504660475905319925895086755774471151890089727930776090169445401259844048317273142069811], [4557234547, 349364318043137479854576449493426376983315472777226775365310579193760250715517761090058069937282741206013319707277840448237966901906357292702335951], [7667001731, 855344863189641492213600127143839128290386097202448105626863527763958015786114563445357087338205788545215994676722500375202243293047596358065835329663]]
ct = "be205fd34ebe59af55ea11fec9aea50197fbf35d5b52c650a6c9563186625e8b6021ba31db538fa4b60c69a42c96ee3bebaba53ac9afa9c3c185d4d0b145bc8251d892c243f1aa4037aeea003714e24c"
iv = "370abc6fce33f812de7b88daaa82e4c4"


## Lattice building and reducing

In [4]:
mat = Matrix(ZZ, 16, 16)
for i in range(16):
    for j in range(16):
        mat[i, j] = hint[j][0] ^ i
# print(mat.T.solve_right(vector([y for x, y in hint])))
mat = mat.augment(matrix.identity(16))
mat = mat.stack(vector([y for x, y in hint] + [0] * 16))
mat_l = mat.LLL()

## Extracting key

In [5]:
key_array = list(map(int, -mat_l[1][16:]))
key_array[0] += mat_l[1][0]
print(key_array)
key = bytearray(key_array)
print(key)
# for v in mat_l:
#     print(v)

[151, 31, 144, 9, 171, 106, 24, 3, 175, 203, 78, 154, 44, 113, 201, 46]
bytearray(b'\x97\x1f\x90\t\xabj\x18\x03\xaf\xcbN\x9a,q\xc9.')


## Decrypting flag

In [6]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad
cip = AES.new(key, AES.MODE_CBC, bytes.fromhex(iv))
flag = cip.decrypt(bytes.fromhex(ct))
print(unpad(flag, 16))


b"HCMUS-CTF{just_because_you're_correct_doesn't_mean_you're_right}"
